In [ ]:
import pandas as pd
import numpy as np
import os
import requests
import env

In [ ]:
# Example usage:
app_token = env.app_token
year_to_retrieve = '2022'
max_req = 2000  # Specify the maximum number of observations to retrieve
